In [1]:
using NBInclude, NLopt
@nbinclude("startup_CAN.ipynb")

C0 is Country1(360.0, 216.0, 480.0, 780.0, 0.0048, NaN, NaN, NaN, NaN, NaN, 0.0, [1.0, 0.0], 0.6, 0.4369434070669348, 0.15943694340706693, NaN, NaN, NaN)
Startup run in 33.13694095611572 secs.


# 1) Find the parameters of the benchmark model

We calibrate our model to determine the values of $\alpha$, $m$ and $q$ (and $\lambda$).

In [2]:
function distance(m1,m2)
    out = 0
    for i in 1:length(m1)
        out += (m1[i] - m2[i]) ^2
    end
    return(out)
end


## compute the objective to minimise
function objective_can(α,m,q0,qa)
    newC = update_Country1(α=α, m=m, q0=q0, qa=qa)
    
    ## simulate
    newV = find_Surplus(newC, reltol=1e-12, findΨ=false)
    unemp = find_Unemp(newV, newC, find_with=false, find_without=false) 
    moments_emp = moments_canada(unemp.real, atol=1e-8)
    
  #  IJulia.clear_output(true)
 #   println("Current parameters: $α, $m, $q0, $qa")
  #  println("Simulated moments: $moments_emp")
    
    distance(moments_emp, moments_target) 
end

## distance minimisation with NLopt
@time parmin = let
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.1 , 0., 0.001, -0.1])
    upper_bounds!(opt, [5. , 1., 0.2, 0.1])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_can(v[1],v[2],v[3],v[4]))
    
    init =  [0.5, 0.2, 0.02, 0.] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    minx
end

Param found with convergence SUCCESS, the value of the objective is 0.0010743751421727638
Parameters: 
 - α= 0.6890330404333282
 - m= 0.15619724389788173
 - q0= 0.05098028312784511
 - qa= 5.5144846333084566e-5
  8.821927 seconds (22.87 M allocations: 1.762 GiB, 4.14% gc time, 83.36% compilation time: <1% of which was recompilation)


4-element Vector{Float64}:
 0.6890330404333282
 0.15619724389788173
 0.05098028312784511
 5.5144846333084566e-5

# 2) Sensitivity: find the parameters with alternative values of $q_0$
Find the parameters for $q_0=0.005, 0.01, 0.025, 0.05, 0.1$.

In [3]:
function find_canada(q0)
    opt = Opt(:LN_BOBYQA, 3)
    lower_bounds!(opt, [1e-5 , 0., -0.1])
    upper_bounds!(opt, [20. , 10., 0.1])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_can(v[1],v[2],q0,v[3]))
    
    init =  [0.5, 0.2, 0. ] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - qa= ", minx[3])
    return(minx, minf)
end

q0_vec = [0.01 0.025 0.05 0.075 0.1]
#[0.005 0.01 0.025 0.05 0.1]
amqa_vec = zeros(3,5)
objectivevalue = zeros(5)
for (i,q0) in enumerate(q0_vec)
    @time amqa_vec[:,i], objectivevalue[i] =  find_canada(q0)
end

println(objectivevalue)
out = vcat(q0_vec,amqa_vec)
show(out)
out

Param found with convergence SUCCESS, the value of the objective is 0.0012304396569548094
Parameters: 
 - α= 0.1876337824599228
 - m= 0.027407404402780222
 - qa= -1.4743789274070752e-5
  0.860377 seconds (6.68 M allocations: 569.862 MiB, 11.55% gc time, 14.12% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.0011502573848390157
Parameters: 
 - α= 0.516900451570764
 - m= 0.07358092016935147
 - qa= 5.883797565856158e-6
  0.178860 seconds (1.54 M allocations: 132.310 MiB, 17.18% gc time)
Param found with convergence XTOL_REACHED, the value of the objective is 0.00110623094440133
Parameters: 
 - α= 0.8546295970511096
 - m= 0.15682740190490604
 - qa= 7.65632286368146e-5
  0.208493 seconds (2.02 M allocations: 177.891 MiB, 11.32% gc time)
Param found with convergence SUCCESS, the value of the objective is 0.0010308885982217375
Parameters: 
 - α= 0.6788267018190596
 - m= 0.2322796134025758
 - qa= 9.546386130991365e-5
  0.418500 seconds (4.58 M allocation

4×5 Matrix{Float64}:
  0.01        0.025      0.05        0.075       0.1
  0.187634    0.5169     0.85463     0.678827    0.641011
  0.0274074   0.0735809  0.156827    0.23228     0.311028
 -1.47438e-5  5.8838e-6  7.65632e-5  9.54639e-5  0.000134273

# 3) Sensitivity: find the parameters with alternative values of $\xi$

In [4]:
function objective_can(α,m,q0,qa,ξ)
    newC = update_Country1(α=α, m=m, q0=q0, qa=qa, ξ=ξ)
    
    ## simulate
    newV = find_Surplus(newC, reltol=1e-12)
    unemp = find_Unemp(newV, newC, find_with=false, find_without=false) 
    moments_emp = moments_canada(unemp.real, atol=1e-8)
    distance(moments_emp, moments_target) 
end

function find_canada(ξ)
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.1 , 0., 0.001, -0.1])
    upper_bounds!(opt, [5. , 1., 0.2, 0.1])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_can(v[1],v[2],v[3],v[4],ξ))
    
    init =  [0.5, 0.2, 0.02, 0.] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    return (minx, minf)
end

ξ_vec = [0 0.25 0.5 0.75 1]
amq_vec = zeros(4,5)
objectivevalue = zeros(5)
for (i,ξ) in enumerate(ξ_vec)
    @time amq_vec[:,i], objectivevalue[i] =  find_canada(ξ)
end

println(objectivevalue)
out = vcat(ξ_vec,amq_vec)
show(out)
out

Param found with convergence SUCCESS, the value of the objective is 0.0010743751421727638
Parameters: 
 - α= 0.6890330404333282
 - m= 0.15619724389788173
 - q0= 0.05098028312784511
 - qa= 5.5144846333084566e-5
  2.266643 seconds (16.80 M allocations: 1.412 GiB, 8.98% gc time, 14.10% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.0009743082882012677
Parameters: 
 - α= 0.6389876521786183
 - m= 0.1456232259616489
 - q0= 0.048345863559521404
 - qa= 3.898079680410013e-5
  3.392385 seconds (30.36 M allocations: 2.570 GiB, 8.90% gc time, 1.16% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.0009245648809944958
Parameters: 
 - α= 0.6355722694738223
 - m= 0.16087729229778894
 - q0= 0.05356494877889571
 - qa= 4.153795428287178e-5
  2.247098 seconds (21.23 M allocations: 1.805 GiB, 8.64% gc time)
Param found with convergence SUCCESS, the value of the objective is 0.0008902679137387339
Parameters: 
 - α= 0.63806541893

5×5 Matrix{Float64}:
 0.0         0.25        0.5        0.75       1.0
 0.689033    0.638988    0.635572   0.638065   0.675166
 0.156197    0.145623    0.160877   0.220206   0.239162
 0.0509803   0.0483459   0.0535649  0.0725979  0.0781959
 5.51448e-5  3.89808e-5  4.1538e-5  6.9224e-5  8.31072e-5